In [1]:
import sys, os, io
import dvc.api
sys.path.append(os.path.abspath(os.path.join('..')))

In [2]:
from scripts.preprocess import Preprocess
from scripts.dataloader import DataLoader

In [3]:
dataloader = DataLoader()
repo = 'https://github.com/Haylemicheal/Pharmaceutical-Sales-Prediction/'
content_train =dvc.api.read(path='../data/clean/merged_data.csv', repo=repo, rev='train-v')
df_merged_train = dataloader.read_csv(io.StringIO(content_train))

In [4]:
df_merged_train.shape

(1017209, 21)

In [5]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

def split_target_feature(df: pd.DataFrame, target_col: str) -> tuple:
        """
        Split the target Column
            Args:
                df: dataset
                target_col: The target column
            return:
                features: The dataframe except the target
                target: The target column
        """
        target = df[[target_col]]
        features = df.drop(target_col, axis=1)
        return features, target

features, target = split_target_feature(df_merged_train, "Sales")

In [6]:
features.head()

,Unnamed: 0,Unnamed: 0_x,Store,DayOfWeek,Date,Customers,Open,Promo,StateHoliday,SchoolHoliday,Unnamed: 0_y,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,0,0,1,5,2015-07-31,555,1,1,0,1,0,c,a,1270.0,9.0,2008.0,0,22.0,2012.0,"Jan,Apr,Jul,Oct"
1,1,1115,1,4,2015-07-30,546,1,1,0,1,0,c,a,1270.0,9.0,2008.0,0,22.0,2012.0,"Jan,Apr,Jul,Oct"
2,2,2230,1,3,2015-07-29,523,1,1,0,1,0,c,a,1270.0,9.0,2008.0,0,22.0,2012.0,"Jan,Apr,Jul,Oct"
3,3,3345,1,2,2015-07-28,560,1,1,0,1,0,c,a,1270.0,9.0,2008.0,0,22.0,2012.0,"Jan,Apr,Jul,Oct"
4,4,4460,1,1,2015-07-27,612,1,1,0,1,0,c,a,1270.0,9.0,2008.0,0,22.0,2012.0,"Jan,Apr,Jul,Oct"


In [7]:
features = features.drop('Date', axis=1)
features = features.drop('PromoInterval', axis=1)
storetype_encoder = preprocessing.LabelEncoder()
storetype = storetype_encoder.fit_transform(features['StoreType'])
features['StoreType'] = storetype

assortment_encoder = preprocessing.LabelEncoder()
assortment = assortment_encoder.fit_transform(features['Assortment'])
features['Assortment'] = assortment

stateHoliday_encoder = preprocessing.LabelEncoder()
stateHoliday = stateHoliday_encoder.fit_transform(features['StateHoliday'])
features['StateHoliday'] = stateHoliday
features.dtypes

Unnamed: 0                     int64
Unnamed: 0_x                   int64
Store                          int64
DayOfWeek                      int64
Customers                      int64
Open                           int64
Promo                          int64
StateHoliday                   int64
SchoolHoliday                  int64
Unnamed: 0_y                   int64
StoreType                      int64
Assortment                     int64
CompetitionDistance          float64
CompetitionOpenSinceMonth    float64
CompetitionOpenSinceYear     float64
Promo2                         int64
Promo2SinceWeek              float64
Promo2SinceYear              float64
dtype: object

In [8]:
from scripts.preprocess import Preprocess
process = Preprocess()
scaled_data = process.min_max_scaler(features)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from tensorflow.keras.layers import Dropout, LSTM
n_steps = 32
model = Sequential()
model.add(LSTM(units =128, activation='relu', return_sequences=True, input_shape = (scaled_data.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units =128, activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units =128, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

history = model.fit(scaled_data, target, epochs = 10, batch_size=64, validation_split=0.1)

2022-09-10 21:58:37.589044: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-10 21:58:37.832654: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64
2022-09-10 21:58:37.832677: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-10 21:58:37.861675: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-10 21:58:38.709

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 18, 128)           66560     
                                                                 
 dropout (Dropout)           (None, 18, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 18, 128)           131584    
                                                                 
 dropout_1 (Dropout)         (None, 18, 128)           0         
                                                                 
 lstm_2 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 1)                 1